In [435]:
#import Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut

In [523]:
#get DataSet
fruits = pd.read_csv("fruit_data_with_colours.csv")

In [493]:
fruits.head(5)

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [494]:
print(min(fruits['width']))
Fruit_name =  dict(zip(fruits.fruit_label.unique(),fruits.fruit_name.unique() ))

5.8


In [495]:
print(type(Fruit_name))
print(Fruit_name)

<class 'dict'>
{1: 'apple', 2: 'mandarin', 3: 'orange', 4: 'lemon'}


In [496]:
#get independent_variable (Features)
x = fruits[['mass','width','height']]

In [497]:
x.head(5)

,mass,width,height
0,192,8.4,7.3
1,180,8.0,6.8
2,176,7.4,7.2
3,86,6.2,4.7
4,84,6.0,4.6


In [498]:
#get dependent_variable (Response)
y = fruits['fruit_label']
y.head()

0    1
1    1
2    1
3    2
4    2
Name: fruit_label, dtype: int64

In [499]:
#Split data 
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42)

In [500]:
#Classifier using KNN
from sklearn.neighbors import KNeighborsClassifier

In [501]:
score = {}
for k in range(1,16,2):
    knn = KNeighborsClassifier(n_neighbors=k)
    #train_data
    knn.fit(x_train,y_train)
    #test_data
    score[k]=knn.score(x_test,y_test)
max_score = max(score,key=score.get)

In [502]:
score

{1: 0.5333333333333333,
 3: 0.6,
 5: 0.5333333333333333,
 7: 0.6,
 9: 0.5333333333333333,
 11: 0.4666666666666667,
 13: 0.4666666666666667,
 15: 0.4}

In [503]:
max_score

3

In [504]:
#Get predicted response 
y_predict = knn.predict(x_test)

In [505]:
y_predict

array([1, 4, 3, 1, 1, 4, 3, 1, 1, 3, 4, 4, 1, 1, 1], dtype=int64)

In [506]:
sample = knn.predict([[120,5.3,8.5]])

In [507]:
sample

array([4], dtype=int64)

In [508]:
sample_name_predicted = Fruit_name[sample[0]]
print(sample_name_predicted)

lemon


# KFold cross validation

In [509]:
kf = KFold(n_splits=3,shuffle=False)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [510]:
for train_index, test_index in kf.split(x):
    print(len(train_index),len(test_index))
    print((train_index),(test_index))

39 20
[20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
39 20
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58] [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
40 19
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58]


In [511]:
def get_score(x_train,x_test,y_train,y_test):
    knn = KNeighborsClassifier(n_neighbors=3)
    #train_data
    knn.fit(x_train,y_train)
    #test_data
    return knn.score(x_test,y_test)

In [512]:
scores=[]
for train_index, test_index in kf.split(x):
    x_train,x_test,y_train,y_test=x.loc[x.index[train_index]],x.loc[x.index[test_index]]\
    ,y.loc[y.index[train_index]],y.loc[y.index[test_index]]
    scores.append(get_score(x_train,x_test,y_train,y_test))

In [513]:
scores

[0.25, 0.2, 0.0]

# KFold with range

In [522]:
mean_scores={}
for k in range(3,8):
    kf = KFold(n_splits=k,shuffle=True)
    scores=[]
    for train_index, test_index in kf.split(x):
        x_train,x_test,y_train,y_test=x.iloc[train_index],x.iloc[test_index]\
        ,y.iloc[train_index],y.iloc[test_index]
        scores.append(get_score(x_train,x_test,y_train,y_test))
    print(scores)
    
    mean_scores[k]=(np.min(scores)+np.max(scores))/2
max_k_score = max(mean_scores,key=mean_scores.get)
print(max_k_score)
mean_scores[max_k_score]

[0.7, 0.7, 0.7894736842105263]
[0.8666666666666667, 0.4666666666666667, 0.6666666666666666, 0.8571428571428571]
[0.6666666666666666, 0.4166666666666667, 0.5833333333333334, 0.75, 0.7272727272727273]
[0.7, 0.7, 0.8, 0.5, 0.6, 0.5555555555555556]
[0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.875, 0.75, 0.75, 0.625]
7


0.75

# LOOCV

In [517]:
loo = LeaveOneOut()
loo.get_n_splits(x)

59

In [518]:
LOOCV_Score=[]
for train_index, test_index in loo.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    LOOCV_Score.append(get_score(x_train,x_test,y_train,y_test))
np.mean(LOOCV_Score)    

0.711864406779661

# standard normal distribution

In [483]:
np.mean(x)

mass      2.597025
width     2.611245
height    3.736867
dtype: float64

In [484]:
np.std(x)

mass      1.0
width     1.0
height    1.0
dtype: float64

In [485]:
np.min(x)

mass      1.0
width     1.0
height    1.0
dtype: float64

In [486]:
x = (x-np.mean(x))/np.std(x)

In [487]:
x.head(10)

,mass,width,height
0,0.529442,1.598690,-0.291397
1,0.309462,1.104854,-0.661922
2,0.236136,0.364100,-0.365502
3,-1.413709,-1.117409,-2.218131
4,-1.450372,-1.364327,-2.292236
5,-1.523699,-1.611245,-2.514552
6,-1.523699,-1.487786,-2.514552
7,-1.597025,-1.611245,-2.736867
8,0.272799,-0.006278,0.079129
9,0.162810,0.364100,-0.513712


In [488]:
x=(x-np.min(x))+1

In [489]:
x.head(10)

,mass,width,height
0,3.126467,4.209936,3.445470
1,2.906488,3.716099,3.074945
2,2.833161,2.975345,3.371365
3,1.183316,1.493836,1.518736
4,1.146653,1.246918,1.444631
5,1.073326,1.000000,1.222315
6,1.073326,1.123459,1.222315
7,1.000000,1.000000,1.000000
8,2.869824,2.604968,3.815996
9,2.759835,2.975345,3.223155


In [490]:
np.min(x)

mass      1.0
width     1.0
height    1.0
dtype: float64